In [2]:
%matplotlib widget

import logging

import httpx
import matplotlib.pyplot as plt
import pandas as pd
import shapely.io
import sqlmodel
from loess.loess_1d import loess_1d

from arpav_ppcv import (
    database as db,
    operations,
)
from arpav_ppcv.config import get_settings
from arpav_ppcv.schemas.base import (
    CoverageDataSmoothingStrategy,
    ObservationDataSmoothingStrategy,
    ObservationAggregationType,
    Season,
)

logging.basicConfig(level=logging.DEBUG)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)
logging.getLogger("matplotlib").setLevel(logging.WARNING)

settings = get_settings()
session = sqlmodel.Session(db.get_engine(settings))
http_client = httpx.Client()

In [9]:
capabilities_response = httpx.get(
    "http://webapp:5001/api/v2/coverages/wms/tas_seasonal_absolute_model_ensemble-rcp26-DJF",
    params={
        "service": "WMS",
        "version": "1.3.0",
        "request": "GetCapabilities"
    }
)

In [10]:
capabilities_response.content

b'<?xml version="1.0" encoding="UTF-8"?>\n<WMS_Capabilities\n        version="1.3.0"\n        updateSequence="2024-06-06T15:38:58.994Z"\n        xmlns="http://www.opengis.net/wms"\n        xmlns:xlink="http://www.w3.org/1999/xlink"\n        xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"\n        xmlns:edal="http://reading-escience-centre.github.io/edal-java/wms"\n        xsi:schemaLocation="http://www.opengis.net/wms http://schemas.opengis.net/wms/1.3.0/capabilities_1_3_0.xsd">\n    <Service>\n        <Name>WMS</Name>\n        <Title>ARPAV test server</Title>\n        <Abstract>Regional climate models and data</Abstract>\n        <KeywordList>\n            <Keyword>meteorology</Keyword>\n            <Keyword> atmosphere</Keyword>\n            <Keyword> climate</Keyword>\n            <Keyword> ocean</Keyword>\n            <Keyword> earth science</Keyword>\n        </KeywordList>\n        <OnlineResource xlink:type="simple" xlink:href="http://thredds:8080/thredds/wms/ensymbc/clipp

In [11]:
from xml.etree import ElementTree as et

In [20]:
ns = {
    "wms": "http://www.opengis.net/wms"
}

In [12]:
root = et.fromstring(capabilities_response.content)

In [13]:
root

<Element '{http://www.opengis.net/wms}WMS_Capabilities' at 0x7de7aaf42660>

In [23]:
root.findall(f"./{{{ns['wms']}}}Capability/{{{ns['wms']}}}Request")

[<Element '{http://www.opengis.net/wms}Request' at 0x7de7aaf43510>]